In [ ]:
import numpy as np
import tensorflow as tf
import re
from collections import Counter
import json
from pprint import pprint
from tensorflow.contrib import learn
import re
import csv
import pickle
import time

# Load Chat Dataset From Pickle

In [ ]:
pkl_file = open('train_x.pkl', 'rb')
train_x = pickle.load(pkl_file)

pkl_file = open('test_x.pkl', 'rb')
test_x = pickle.load(pkl_file)

pkl_file = open('val_x.pkl', 'rb')
val_x = pickle.load(pkl_file)

pkl_file = open('train_y.pkl', 'rb')
train_y = pickle.load(pkl_file)

pkl_file = open('test_y.pkl', 'rb')
test_y = pickle.load(pkl_file)

pkl_file = open('val_y.pkl', 'rb')
val_y = pickle.load(pkl_file)

# Util Functions

In [ ]:
#Write To csv File
def file_writer(file_name,row_):
    with open(file_name, 'a') as f:
        spamwriter = csv.writer(f, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(row_ )
        
#Batching
def get_batches(x, y, batch_size=100):
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]
        



# Construct Possible Hyperparameter List

In [ ]:
lstm_size_list = [16,16,32,64]
lstm_layers_list = [1]
batch_size_list = [128,32,64,128]
n_epochs_list = [10,15,20]

all_hyperpara_list = []
for a in lstm_size_list:
    for b in lstm_layers_list:
        for c in batch_size_list:
            for d in n_epochs_list:
                all_hyperpara_list.append((a,b,c,d))

# Start Hyperparameter Tuning

In [ ]:
hyperpara_index = 0 #Hyperparameter S.No.

for hyperameter_tuple in all_hyperpara_list: 
    
    #Current set of Hyperparmeters
    lstm_size = hyperameter_tuple[0]
    lstm_layers = hyperameter_tuple[1]
    batch_size = hyperameter_tuple[2]
    n_epochs = hyperameter_tuple[3]
    file_writer('hyper_'+str(hyperpara_index)+'.csv',['lstm_size= '+str(lstm_size), 'lstm_layers= '+str(lstm_layers),'batch_size= '+str(batch_size),'n_epochs= '+str(n_epochs)])    

    #Reset Graph 
    from tensorflow.python.framework import ops
    ops.reset_default_graph()

    #Placeholder
    X = tf.placeholder(tf.float32, [None, None, 300], name = 'inputs')
    Y = tf.placeholder(tf.float32, [None, 1], name = 'labels')

    #Build Network
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm]*lstm_layers)
    initial_state = cell.zero_state(batch_size, tf.float32)
    outputs, final_state = tf.nn.dynamic_rnn(cell, X, initial_state = initial_state)
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1],1, activation_fn=tf.tanh)

    #Optimisation
    loss = tf.reduce_mean(tf.square(Y - predictions))
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

    #Accuracy
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.float32), Y)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    #Start Time
    start_time = time.time()

    #Temp Variables
    x_ = [] #Accounts for batch size
    loss_count = 0
    train_loss = []
    val_loss = []

    #START SESSION:
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    #For every Epoch
    for e in range(n_epochs):
        state = sess.run(initial_state)
        batch_index = 1 #represents index of batch
        
        #For every batch
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {X: x, Y: np.reshape(y,(len(y),1)), initial_state: state}
            state, loss_,  _ = sess.run([final_state, loss, optimizer], feed_dict=feed)
            
            #Calculate TrainLoss & ValidationLoss for building Loss graph
            if batch_index%50==0:
                #Train Loss:
                print("Epoch: {}/{}".format(e, n_epochs),"Iteration: {}".format(batch_index),"Train loss: {:.3f}".format(loss_))
                train_loss.append(loss_)
                loss_count = loss_count + 1
                x_.append(loss_count)

                #Validation loss: 
                #Steps =  - Calculate loss for every batch in validation set
                #         - Take the mean of all batch as final validation loss
                temp_val_loss = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size): 
                    feed = {X: x,Y: np.reshape(y,(len(y),1)),initial_state: val_state}
                    loss_, val_state = sess.run([loss, final_state], feed_dict=feed)
                    temp_val_loss.append(loss_)
                val_loss.append(np.mean(temp_val_loss))  
                
            #Calculate TrainAcc & ValidationAcc & TestAcc for writing in file
            if batch_index%100==0:
                #Validation Acc:
                #Steps =  - Calculate loss for every batch in validation set
                #         - Take the mean of all batch as final validation Accuracy
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {X: x,Y: np.reshape(y,(len(y),1)),initial_state: val_state}

                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.5f}".format(np.mean(val_acc)))
                file_writer('hyper_'+str(hyperpara_index)+'.csv',['Epoch= '+str(e),'Validation Acc= '+str(np.mean(val_acc)) ])

                #Train Acc:
                train_acc = []
                train_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
                    feed = {X: x,Y:  np.reshape(y,(len(y),1)),initial_state: train_state}

                    batch_acc, train_state = sess.run([accuracy, final_state], feed_dict=feed)
                    train_acc.append(batch_acc)
                print("Train accuracy: {:.5f}".format(np.mean(train_acc)))
                file_writer('hyper_'+str(hyperpara_index)+'.csv',['Epoch= '+str(e),'Train Acc= '+str(np.mean(train_acc)) ])

                #Test Acc: 
                test_acc = []
                test_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
                    feed = {X: x,Y: np.reshape(y,(len(y),1)),initial_state: test_state}

                    batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
                    test_acc.append(batch_acc)
                print("Test accuracy: {:.5f}".format(np.mean(test_acc)))
                file_writer('hyper_'+str(hyperpara_index)+'.csv',['Epoch= '+str(e),'Test Acc= '+str(np.mean(test_acc)) ])

            #represents index of batch
            batch_index +=1
    
    #All epochs completed
    print('Training Completed')

    #Test Acc(after complete training): 
    test_acc = []
    #test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {X: x,Y: np.reshape(y,(len(y),1))}

        batch_acc = sess.run([accuracy], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.5f}".format(np.mean(test_acc)))
    file_writer('hyper_'+str(hyperpara_index)+'.csv',['Epoch= '+str(e),'Final Test Acc= '+str(np.mean(test_acc)) ])

    #Total Time Taken for this hyperparameter set
    time_taken = time.time()-start_time 

    #Build Graph:
    import matplotlib.pyplot as plt
    plt.plot(x_,train_loss,'g',x_,val_loss,'b')
    plt.savefig("figure"+str(hyperpara_index)+".png")
    plt.show()

    #Write Time_Taken, Min_Val_loss
    file_writer('hyper_'+str(hyperpara_index)+'.csv',['time_taken= '+str(time_taken),'min val loss= '+str(np.min(val_loss)) ])

    #Update Hyperparameter S.No.
    hyperpara_index+=1